In [9]:
import sqlite3
import pandas as pd
import logging

def ingest_db(df,table_name,engine):
    '''This fuction will ingest dataframe in database table'''
    df.to_sql(table_name,con = engine,if_exists = "replace",index =False)
    #if need to add data every 15 minutes then append 

logging.basicConfig(
   filename="logs/get_vendor_summary.log",
   level= logging.DEBUG,
   format= "%(asctime)s - %(levelname)s - %(message)s",
   filemode= "a"
)

def create_vendor_summary(conn):
    #This function will merge the different tables to get the overall vendor 
    # summary and addimg new columns in the resultant data
    vendor_sales_summary = pd.read_sql_query('''WITH freight_summary AS (
        SELECT 
            VendorNumber,
            SUM(freight) AS Total_Freightcost
        FROM Vendor_invoice
        GROUP BY VendorNumber
    ),
    purchase_summary AS (
        SELECT
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Volume,
            pp.Price AS Actual_price,
            SUM(p.Quantity) AS Total_purchaseQuantity,
            SUM(p.Dollars) AS Total_purchaseDollars
        FROM Purchases AS p
        JOIN purchase_prices AS pp
            ON p.Brand = pp.Brand
        WHERE p.PurchasePrice > 0
        GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.PurchasePrice, pp.Volume, pp.Price
    ),
    sales_summary AS (
        SELECT
            VendorNo,
            Brand,
            SUM(SalesDollars) AS Totalsales_Dollars,
            SUM(SalesPrice) AS Totalsales_price,
            SUM(SalesQuantity) AS Totalsales_quantity,
            SUM(ExciseTax) AS Total_Excisetax
        FROM sales
        GROUP BY VendorNo, Brand
    )
    SELECT
        ps.VendorNumber,
        ps.VendorName,
        ps.Brand,
        ps.Description,
        ps.Actual_price,
        ps.PurchasePrice,
        ps.Volume,
        ps.Total_purchaseQuantity,
        ps.Total_purchaseDollars,
        ss.Totalsales_quantity,
        ss.Totalsales_price,
        ss.Totalsales_Dollars,
        ss.Total_Excisetax,
        fs.Total_Freightcost
    FROM purchase_summary AS ps
    LEFT JOIN sales_summary AS ss
        ON ps.VendorNumber = ss.VendorNo
        AND ps.Brand = ss.Brand
    LEFT JOIN freight_summary AS fs
        ON ps.VendorNumber = fs.VendorNumber
    ORDER BY ps.Total_purchaseDollars DESC''', conn)
  
    return vendor_sales_summary

def clean_data(df):
    #this function will clean the data
    #changing data type to float
    df['Volume'] = df['Volume'].astype('float')

    #filling missing values with 0
    df.fillna(0,inplace=True)

    #Remove spaces from categorical columns
    df['VendorName'] = df['VendorName'].str.strip()
    
    #Creating new columns for better analysis
    df['GrossProfit'] = df['Totalsales_Dollars']-df['Total_purchaseDollars']
    df['Profitmargin']= (df['GrossProfit']/df['Total_purchaseDollars'])*100
    df['StockTurnover'] = df['Totalsales_quantity']/df['Total_purchaseQuantity']
    df['SalesPurchaseRatio'] = df['Totalsales_Dollars']/df['Total_purchaseDollars']

    return df  

if __name__ == '__main__':
    Conn = sqlite3.connect('inventory.db')
    logging.info('Creating Vendor Summary table...')  
    summary_df = create_vendor_summary(Conn) 
    print(summary_df.head())

    logging.info('Ingesting data...')
def ingest_db(clean_func, summary_func, conn):
    # Step 1: Call the summary function to get the raw DataFrame
    df = summary_func(conn)

    # Step 2: Clean the DataFrame using the cleaning function
    cleaned_df = clean_func(df)

    # Step 3: Save the cleaned DataFrame to the database
    table_name = "vendor_sales_summary"
    cleaned_df.to_sql(table_name, con=conn, if_exists="replace", index=False)  
logging.info('completed')

   VendorNumber                   VendorName  Brand              Description  \
0          1128  BROWN-FORMAN CORP             1233  Jack Daniels No 7 Black   
1          4425        MARTIGNETTI COMPANIES   3405    Tito's Handmade Vodka   
2         17035  PERNOD RICARD USA             8068         Absolut 80 Proof   
3          3960  DIAGEO NORTH AMERICA INC      4261   Capt Morgan Spiced Rum   
4          3960  DIAGEO NORTH AMERICA INC      3545          Ketel One Vodka   

   Actual_price  PurchasePrice Volume  Total_purchaseQuantity  \
0         36.99          26.27   1750                  145080   
1         28.99          23.19   1750                  164038   
2         24.99          18.24   1750                  187407   
3         22.99          16.17   1750                  201682   
4         29.99          21.89   1750                  138109   

   Total_purchaseDollars  Totalsales_quantity  Totalsales_price  \
0             3811251.60             142049.0         672819.